# Introduction

The goal of this project is to analyze non-normal distributed data of open source GitHub projects from a dataset provided by .

The project should show an ability to explore data, handle ambiguity, and weave insights into a narrative. It should have well-formed observations and understanding of the nuances of the data.

In [1]:
# Import required modules
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import seaborn as sns
from datetime import timedelta

# Data Extraction

The dataset is provided from two files "usage.csv" and "projects.csv". They are selected usage records of open source GitHub projects.

In [2]:
usage = pd.read_csv("usage.csv")

# To view initial structure of dataset
usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566094 entries, 0 to 566093
Data columns (total 6 columns):
id            566094 non-null object
actor_id      444652 non-null object
project_id    566094 non-null object
account_id    566094 non-null object
started_at    566094 non-null object
ended_at      566094 non-null object
dtypes: object(6)
memory usage: 25.9+ MB


There are 566094 observations/rows in usage and 6 columns/variables. Every column, except for the actor_id has non-null values.

In [3]:
project = pd.read_csv("projects.csv")
project.columns=['project_id','project_size']
project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12907 entries, 0 to 12906
Data columns (total 2 columns):
project_id      12907 non-null object
project_size    12906 non-null float64
dtypes: float64(1), object(1)
memory usage: 201.8+ KB


The initial view of the projects data file shows that there is 1 null value.

In [4]:
# Merging two datasets
merged = pd.merge(usage, project, on='project_id', how='outer')

In [5]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566094 entries, 0 to 566093
Data columns (total 7 columns):
id              566094 non-null object
actor_id        444652 non-null object
project_id      566094 non-null object
account_id      566094 non-null object
started_at      566094 non-null object
ended_at        566094 non-null object
project_size    563510 non-null float64
dtypes: float64(1), object(6)
memory usage: 34.6+ MB


# Data Wrangling/Munging

In [6]:
merged['started_at'] = pd.to_timedelta('00:' + merged['started_at'], errors='coerce') 
merged['ended_at'] = pd.to_timedelta('00:' + merged['ended_at'], errors='coerce')   

Due to the non-powerful nature of my tablet-laptop, I have subsetted the data to 384 observations. This number was determined by calculating the sample size with a population of 566,094 (the total number of observations), a confidence level of 95%, and a margin of error of 5%. Since I am limiting the dataset, I also removed all null values.

In [7]:
no_na = merged[pd.notnull(merged['actor_id'])]
no_na = no_na[pd.notnull(merged['project_size'])]
small = no_na.sample(384)
small = small.reset_index(drop=True)

C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [10]:
small.head()

,id,actor_id,project_id,account_id,started_at,ended_at,project_size
0,bf8cab9f9149bab845afe66325414292701b11c61861ff...,c46477d6f6c30a8beb1dfded05831e088eca300781df6d...,da4499ef83cdf572787f0e1ae3e81684116a3e296be777...,036eadeac583ba630ec8d7a22a3b495d2f9e9268c28e0f...,00:42:31.300000,01:35:22,7764.0
1,41646c5e9b8a975b2f8640a0676c285fe4874752210a69...,c77914feecb11bfea419d572c4ff7c2576d32a206f5221...,0f82fe8166494df390f38385cd2d1691a4374fe1c0d6c7...,09fdd9137c9f5885d4a413864e9839c5d98fefb9588715...,00:40:53.600000,00:44:01.900000,192729.0
2,382bf302eb86c807626e77dd6ea29299fdbf496a0d1d1f...,779d9b3f3bb0396df4adc385bc8de0487dd32b4acb55c6...,d452291abaae5727935e6c80768bb51fd8ceaf6c537dd1...,203ea8da668de8d4cd68cfe51dcf998bbc7bea8b1b7206...,00:01:12.700000,00:01:18.700000,29.0
3,4214f67e79d34ad0cea7be77bae0ab9900f59650718c09...,467a3f755fafd35efc28ff5ebacd193c61c5e9ba388a84...,f09396375c9a706d291c6264cdcfd441182d9d422f2b7d...,d2fefe1774b77114507fdce4a9c63cf13a6bede7afff9d...,00:57:02.500000,00:58:00.200000,71.0
4,fed1962fa12855cf01eeaee25a726d589d810a3326c9ab...,25aab5baedbe1b53d5312f713dfa32bcc18f944af914d9...,79c6bbae98854c8f2fac9b78411575db8a922af71dcb24...,1e9deba2f445989233c8397686ece4c92c93d848a943f5...,00:14:33.900000,00:15:14.100000,114.0


Since the imported data only measured time in minutes and dates, events such as the start time being after the end time occured. This is not realisticall possible. Therefore the next code segment checks that the laws of time have not been broken, if so, it makes adds an hour until the end date is after the start date. This results in a more accurate and realistic time of, for example 2 minutes, than 23 hours and 58 minutes.

In [9]:
i=0
while i < len(small):
    try:
        while small['ended_at'][i]<small['started_at'][i]:
            small['ended_at'][i] = small['ended_at'][i]+timedelta(0,60*60)
    except TypeError:
        pass
    i += 1

C:\Users\Kevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [11]:
# Calculate time_of_usage

calc = small.apply(lambda row: row['ended_at'] - row['started_at'], axis = 1)

small['time_of_usage'] = calc

# Check / Test
small

,id,actor_id,project_id,account_id,started_at,ended_at,project_size,time_of_usage
0,bf8cab9f9149bab845afe66325414292701b11c61861ff...,c46477d6f6c30a8beb1dfded05831e088eca300781df6d...,da4499ef83cdf572787f0e1ae3e81684116a3e296be777...,036eadeac583ba630ec8d7a22a3b495d2f9e9268c28e0f...,00:42:31.300000,01:35:22,7764.0,00:52:50.700000
1,41646c5e9b8a975b2f8640a0676c285fe4874752210a69...,c77914feecb11bfea419d572c4ff7c2576d32a206f5221...,0f82fe8166494df390f38385cd2d1691a4374fe1c0d6c7...,09fdd9137c9f5885d4a413864e9839c5d98fefb9588715...,00:40:53.600000,00:44:01.900000,192729.0,00:03:08.300000
2,382bf302eb86c807626e77dd6ea29299fdbf496a0d1d1f...,779d9b3f3bb0396df4adc385bc8de0487dd32b4acb55c6...,d452291abaae5727935e6c80768bb51fd8ceaf6c537dd1...,203ea8da668de8d4cd68cfe51dcf998bbc7bea8b1b7206...,00:01:12.700000,00:01:18.700000,29.0,00:00:06
3,4214f67e79d34ad0cea7be77bae0ab9900f59650718c09...,467a3f755fafd35efc28ff5ebacd193c61c5e9ba388a84...,f09396375c9a706d291c6264cdcfd441182d9d422f2b7d...,d2fefe1774b77114507fdce4a9c63cf13a6bede7afff9d...,00:57:02.500000,00:58:00.200000,71.0,00:00:57.700000
4,fed1962fa12855cf01eeaee25a726d589d810a3326c9ab...,25aab5baedbe1b53d5312f713dfa32bcc18f944af914d9...,79c6bbae98854c8f2fac9b78411575db8a922af71dcb24...,1e9deba2f445989233c8397686ece4c92c93d848a943f5...,00:14:33.900000,00:15:14.100000,114.0,00:00:40.200000
5,3769fe3cab948de7f4fedc5dad1828c080cfc5eeed209e...,e9d3a9cb09cea8812bd1d93cffa6bfbe700f8afe32c482...,dd7194d27ea46f495834074d82c3df8d000664a0408d54...,fbd071c0fc3a9043967f3986672edc0b71e2fa60c17551...,00:11:52,00:12:33.700000,404.0,00:00:41.700000
6,fd556e301ec5c4f650d7d7cf9db5f5c38c5f96a90d581a...,4a80bd9ca1525c36321ad59fba564c65eeb286cfc32c65...,21ddd450951d78d4f634feb1f0a4ce4a228b833a8a360d...,9e6cebcbce5063657ba3491e92303acc9844c214e421af...,00:33:01.600000,01:04:14.500000,123831.0,00:31:12.900000
7,dd803a121e1f906efda6e85ae7ca51ba9acf9315ff3075...,ee16415227dc8c75613227f08c64144403621ae586a7a6...,5b36c75e22078eeb6e40d11fae73f6e3bd36b6c0821323...,99dbeb25ae7ec8be31adbd5f96787480ff1782a6af0179...,00:01:38.600000,00:02:54,6110.0,00:01:15.400000
8,0e7af1c84ab57e2adb9cf8fdce0da4a5b36881719fba80...,d848dc9c39091b96338035ca5c260295f7621065b700ec...,77595c3dd3ccf75332281ea20622f0711892fb7ac65634...,e9092495b57f81055d7fba885515b1478285d68683438c...,00:04:31,00:04:43.900000,51806.0,00:00:12.900000
9,9347e5b962f1142591129ccfc16c7cb6bd0939e7cd6ab8...,1deadbaa41de4d4e7cb162fc3923f82c98f4861a30d3b1...,81022db9212a613939af8bbad4ef8df621ba38dd448176...,777f22203ccc478c4068189542c50f490868cd66652dcc...,00:37:22,00:37:38.400000,2150.0,00:00:16.400000


In [ ]:
merged

# Section 1 Questions

#1 Identify the top 10 actors - by total usage, by accounts built against, and by projects built against

In [ ]:
# By Total Usage

In [15]:
# By Acccounts Built Against
small.groupby(by=['actor_id']).sum()

,project_size
actor_id,
00bdf40f70f54ea4a464dc2f34b853f9c78e70cd55affb6c930206d6004762d3,23729.0
017a7c8cf1340e89f3ec23e902606430bd911804bbb3c3820091650344d6f69f,60907.0
04477b6a449493f7d060508eecdf42c20f6f0f7f7fb7ee97f96a326dcbd5d07f,20376.0
059b10950ee999efb6a8b69308586198994871484c92c5a3d382b92596f0ae4e,169312.0
062ec79a5705b77e62504dfdb8e171731ebd1db9499a9e7e79cea37f468956dd,26.0
069c778722156b659771450cb6a92987e6decc2dd324e6a78a8ddd86c9e71b89,111.0
074da42d4da865db011f49ad58cc40e24aa2b2aa2e5bb1c789ad45cf334dc8a5,49172.0
0756faeebacbd0f8e000815bef3554014bc1bf0ffcb38e7a46eb9f9be897e40f,3.0
09a31247440b37b5db2f82a156138395ee872eac459f25151c3928762969a658,165.0


In [16]:
# By Projects Built Against

#2 For each ranking, also provide the scalar amounts

#3 Provide a five-number summary of usage for each week, and a five-number summary of total usage per account for each week

In [ ]:
# The provided datasets do not specify weeks so it is not possible to know if these events
# happened on different weeks or the same week

merged.describe()

#4 Provide some comments on the following: Where can a t-test be applied to these collections? Provide some examples of where it can and cannot, insights on why, and other implications

In [ ]:
A t-test can be us

#5 Provide an analysis that shows the relationship between the following variables, along with insights drawn out and comments on your choices:
- Total usage per account
- Unique Actors per account
- Two other variables

In [ ]:
rho, pval = stats.spearmanr(small)

pvals = pd.DataFrame(pval)
#pvals = pvals.rename(index={0: 'outlook',1: 'net_income',2: 'ft_employee',3: 'div_yield',4: 'five_year',5: 'ten_year',6: 'alcohol',7: 'gambling',8: 'tobacco',9: 'controversial_weapons',10: 'small_arms',11: 'military_contracting',12: 'coal',13: 'dont_use',14: 'dont_trust',15: 'assets',16: 'market_value',17: 'revenue',18: 'net_per_emp'})
#round(pvals, 3)
pval

In [ ]:
# Fill in blanks
merged2["ft_employee"].fillna(merged2["employees"], inplace=True)
merged2["sector"].fillna(merged2["sector_f"], inplace=True)
merged2["industry"].fillna(merged2["industry_f"], inplace=True)

# Removing Duplicate Column
merged2 = merged2.drop('employees', 1)
merged2 = merged2.drop('sector_f', 1)
merged2 = merged2.drop('industry_f', 1)

In [ ]:
# Calculate Net Income Per an Employee

calc = merged2.apply(lambda row: row['net_income'] / row['ft_employee'], axis = 1)

merged2['net_per_emp'] = calc

# Check / Test
merged2

In [ ]:
# Correlation and P-Values

matrix = calcdf
matrix = matrix.drop('date',1)
matrix = matrix.drop('ticker',1)
matrix = matrix.drop('security',1)
matrix = matrix.drop('sector',1)
matrix = matrix.drop('industry',1)
matrix = matrix.drop('adult',1)

rho, pval = stats.spearmanr(matrix)
# print('stats.spearmanr - cor:\n', rho)
# print('stats.spearmanr - pval\n', pval)

#pval

# round(pd.DataFrame(pval), 3)

pvals = pd.DataFrame(pval, columns=['outlook','net_income','ft_employee','div_yield','five_year','ten_year','alcohol','gambling','tobacco','controversial_weapons','small_arms','military_contracting','coal','dont_use','dont_trust','assets','market_value','revenue','net_per_emp'])
pvals = pvals.rename(index={0: 'outlook',1: 'net_income',2: 'ft_employee',3: 'div_yield',4: 'five_year',5: 'ten_year',6: 'alcohol',7: 'gambling',8: 'tobacco',9: 'controversial_weapons',10: 'small_arms',11: 'military_contracting',12: 'coal',13: 'dont_use',14: 'dont_trust',15: 'assets',16: 'market_value',17: 'revenue',18: 'net_per_emp'})
round(pvals, 3)

In [ ]:
mask = np.zeros_like(pvals)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    p2 = sns.heatmap(pvals, mask=mask, square=True, cmap="PuOr")

# sns.heatmap(pvals, square=True)

In [ ]:
# Not normally distributed so Spearman
cm = matrix.corr(method='spearman')
cm

In [ ]:
sns.heatmap(cm, square=True)
plt.yticks(rotation=0)
plt.xticks(rotation=90)

In [ ]:
items = ['outlook','net_income','ft_employee','div_yield','five_year','ten_year','alcohol','gambling','tobacco','controversial_weapons','small_arms','military_contracting','coal','dont_use','dont_trust','assets','market_value','revenue','net_per_emp']
arr = []

for i in items:
    for j in items:
        if pvals.loc[i,j] <= 0.05 and (cm.loc[i,j] > 0.5 or cm.loc[i,j] < -0.5) and i != j and [j,i, pvals.loc[j,i], cm.loc[j,i]] not in arr:
            arr.append([i,j, pvals.loc[i,j], cm.loc[i,j]])
        

arr = pd.DataFrame(arr,columns=['1','2','P-Value','R'])
arr.sort_values('R', ascending = True)

In [ ]:
five_year_df = calcdf[calcdf.outlook.notnull()]
five_year_df = five_year_df[five_year_df.five_year.notnull()]
five_year_df

x, y = five_year_df.outlook, five_year_df.five_year
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
line = x*slope + intercept
five_corr = plt.plot(x,y, 'x', x, line, color = 'black')
plt.xlabel('% Positive Business Outlook')
plt.ylabel('% Return on Investment')
plt.show()

print('Line Equation: Five Year Return = ({})*Outlook +{}'.format(slope,intercept))
print('Line of Best Fit Correlation: {}'.format(r_value))

In [ ]:
ten_year_df = calcdf[calcdf.outlook.notnull()]
ten_year_df = ten_year_df[ten_year_df.ten_year.notnull()]
ten_year_df

x, y = ten_year_df.outlook, ten_year_df.five_year
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
line = x*slope + intercept
five_corr = plt.plot(x,y, 'x', x, line, color = 'black')
plt.xlabel('% Positive Business Outlook')
plt.ylabel('% Return on Investment')
plt.show()

print('Line Equation: Ten Year Return = ({})*Outlook +{}'.format(slope,intercept))
print('Line of Best Fit Correlation: {}'.format(r_value))

In future data collections, the information will be gathered but the date will change. Thus, it will be possible to develop a time series forcast.

In [ ]:
X = calcdf[np.isfinite(calcdf['outlook'])]
X = X[np.isfinite(X['net_income'])]
X = X[np.isfinite(X['ft_employee'])]
X = X[np.isfinite(X['div_yield'])]
X = X[np.isfinite(X['five_year'])]
X = X[np.isfinite(X['assets'])]
X = X[np.isfinite(X['market_value'])]
X = X[np.isfinite(X['revenue'])]

Y = np.array(X['five_year'])
X = np.array(X.drop(['ticker','security','sector','industry','alcohol','adult','gambling','tobacco','controversial_weapons',
                          'small_arms','military_contracting','coal','dont_use','dont_trust','five_year','ten_year'],1))

In [ ]:
from sklearn import preprocessing, cross_validation
from sklearn.linear_model import LinearRegression

In [ ]:
X = preprocessing.scale(X)

In [ ]:
X

In [ ]:
# Checking Lengths Match
print(len(X), len(Y))

X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=0.2)

clf = LinearRegression(n_jobs = -1)
clf.fit(X_train, Y_train)

squarederror = clf.score(X_test, Y_test)

print(squarederror)

# Predicted
forecast = clf.predict(X)

print(forecast)